In [4]:
%load_ext autoreload

In [5]:
%autoreload 2

import pandas as pd
import numpy as np
from fbi_nibrs import *
import time
from states import *

# # state = states_av[1]
# path = "/root/Work/cdc_project/FBI_UCR/data/compressed"
# files = os.listdir(path)
# files.sort()
# # years = ["2012"]#, "2016", "2017", "2018", "2019", "2020"]

# stats = {
#     "year": [],
#     "state": [],
#     "incidents": [],
#     "CM_Other": [],
#     "CM_Physical_Abuse": [],
#     "CM_Neglect": [],
#     "CM_Sexual_Abuse": [],
#     "IPV_Other": [],
#     "IPV_Physical_Abuse": [],
#     "IPV_Neglect_or_Emotional_Abuse": [],
#     "IPV_Sexual_Abuse": [],
# }

# outcomes = pd.DataFrame()
# corruped_files = []

# for state in states_av:
#     files_state = [path + '/' + file for file in files if state in file]
#     print(f"\n {state}", end=" ")
#     for zip_file in files_state:
#         if zip_file[-4:] != ".zip":
#             os.remove(zip_file)
#         print(f"{zip_file[-8:-4]}", end=" ")
#         temp_folder = "/root/Work/cdc_project/FBI_UCR/data/temp/"
#         temp_folder = extract_files(zip_file, temp_folder, state, keep_files=False)
#         try:
#             df = filter_data(temp_folder)
#             stats["year"].append(zip_file[-8:-4])
#             stats["state"].append(state)
#             stats["incidents"].append(len(df))
#             if len(df) == 0:
#                 stats["total_CM_Neglect"].append(0)
#                 stats["total_CM_Sexual_Abuse"].append(0)
#                 stats["total_IPV"].append(0)
#                 stats["total_IPV_Physical_Abuse"].append(0)
#                 stats["total_IPV_Neglect_or_Emotional_Abuse"].append(0)
#                 stats["total_IPV_Sexual_Abuse"].append(0)
#             else:   
#                 # Tag the type of incident
#                 df["OUTCOME"] = df.apply(lambda x : get_crime_type(x.RELATIONSHIP_ID, x.OFFENSE_TYPE_ID), axis=1)
#                 outcomes = outcomes.append(df)
#                 counts = df.OUTCOME.value_counts()
#                 for key in stats.keys():
#                     if key in ["year", "state", "incidents"]:
#                         continue
#                     if key in counts.index:
#                         stats[key].append(counts[key])
#                     else:
#                         stats[key].append(0)
#         except:
#             temp_folder = remove_files(temp_folder, state)
#             print(u'\u2717', end=" ")
#             corruped_files.append(zip_file)
#             stats["year"].pop()
#             stats["state"].pop()
#             stats["incidents"].pop()
#             continue
#         # break
#         temp_folder = remove_files(temp_folder, state)
#         print(u'\u2713', end=" ")

# stats_df = pd.DataFrame(stats)
# stats_df.to_csv("/root/Work/cdc_project/FBI_UCR/data/stats.csv", index=False)
# outcomes.to_csv("/root/Work/cdc_project/FBI_UCR/data/outcomes.csv", index=False)
# corruped_files_series = pd.Series(corruped_files)
# corruped_files_series.to_csv("/root/Work/cdc_project/FBI_UCR/data/corruped_files.csv", index=False)

In [6]:
states = {value:key for key, value in states_name_dict.items()}

stats_df = pd.read_csv("/root/Work/cdc_project/FBI_UCR/data/stats.csv")

agency_participation = pd.read_csv("/root/Work/cdc_project/FBI_UCR/data/ucr_participation_1960_2020.csv")
agency_participation.rename(columns={"state_name": "state", "data_year": "year"}, inplace=True)
agency_participation.state = agency_participation.state.apply(lambda x: states[x] if x in states.keys() else np.nan)
agency_participation.dropna(subset=["state"], inplace=True)

#  Remome observations outisde the range of the years [2000, 2020]
stats_df = stats_df[(stats_df.year >= 2000) & (stats_df.year <= 2020)]
agency_participation = agency_participation[(agency_participation.year >= 2000) & (agency_participation.year <= 2020)]

# # Identify the states with consecutive years of data
states_consecutive = stats_df.groupby("state").agg({"year": lambda x: x.diff().sum() == 20})
states_consecutive = states_consecutive[states_consecutive.year ].index.tolist()

# Remove the states without consecutive years of data
stats_df = stats_df[stats_df.state.isin(states_consecutive)]
agency_participation = agency_participation[agency_participation.state.isin(states_consecutive)]

agency_participation.head()

stats_df = stats_df.merge(agency_participation[["year", "state", "total_population", "nibrs_population"]], on=["state", "year"], how="left")
stats_df["pct_population_covered"] = stats_df["nibrs_population"] / stats_df["total_population"]


stats_df.loc[:, "incidents_normalized"] = stats_df.incidents / stats_df.nibrs_population # normalize by population
stats_df.loc[:, "CM_normalized"] = stats_df[ ["CM_Other", "CM_Physical_Abuse", "CM_Neglect", "CM_Sexual_Abuse"]].sum(axis=1) / stats_df.nibrs_population # normalize by population
stats_df.loc[:, "IPV_normalized"] = stats_df[["IPV_Other", "IPV_Physical_Abuse", "IPV_Neglect_or_Emotional_Abuse", "IPV_Sexual_Abuse"]].sum(axis=1) / stats_df.nibrs_population # normalize by population

stats_df.to_csv("/root/Work/cdc_project/FBI_UCR/data/stats_with_population.csv", index=False)

In [7]:
# States 80% of the population are covered by NIBRS
states80 = []
for g, d in stats_df.groupby("state"):
    if d.pct_population_covered.min() >= 0.80:
        states80.append(g)

In [14]:
stats_df[stats_df.state.isin(states80)].to_csv("/root/Work/cdc_project/FBI_UCR/data/stats_80_pct_population_covered.csv", index=False)

In [9]:
outcomes = pd.read_csv("/root/Work/cdc_project/FBI_UCR/data/outcomes.csv")

In [10]:
outcomes

,VICTIM_ID,INCIDENT_ID,AGE_NUM,SEX_CODE,RACE_ID,ETHNICITY_ID,RESIDENT_STATUS_CODE,RELATIONSHIP_ID,OFFENSE_TYPE_ID,AGENCY_ID,OUTCOME
0,129987,129987,27.0,F,1.0,3.0,R,26,27,106,IPV_Physical_Abuse
1,130095,130095,25.0,M,1.0,3.0,R,21,27,106,IPV_Physical_Abuse
2,130076,130076,23.0,M,2.0,3.0,R,3,27,106,IPV_Physical_Abuse
3,129840,129840,29.0,F,2.0,3.0,R,3,27,106,IPV_Physical_Abuse
4,129882,129882,29.0,U,0.0,3.0,R,21,27,106,IPV_Physical_Abuse
...,...,...,...,...,...,...,...,...,...,...,...
1502516,147236858,133987907,26.0,F,1.0,2.0,R,3,27,21928,IPV_Physical_Abuse
1502517,143027775,130266763,16.0,F,1.0,2.0,R,3,36,21928,IPV_Sexual_Abuse
1502518,144265831,131363803,39.0,F,1.0,2.0,R,21,36,21928,IPV_Sexual_Abuse
1502519,147203542,133962221,45.0,F,1.0,2.0,R,3,27,21928,IPV_Physical_Abuse


In [123]:
from  zipfile import ZipFile

path = "/root/Work/cdc_project/FBI_UCR/data/compressed"
files = os.listdir(path)
files.sort()
years = [str(i) for i in range(2000, 2021)]


outcomes = pd.DataFrame()
corruped_files = []

agencies = pd.DataFrame()

for state in states80:
    files_state = [path + '/' + file for file in files if state in file and file[-8:-4] in years]
    print(f"\n {state}", end=" ")
    for zip_file in files_state:
        if zip_file[-4:] != ".zip":
            os.remove(zip_file)
        z = ZipFile(zip_file)
        agency_files = [z_i for z_i in  z.namelist() if "agencie" in z_i.lower()]
        agency_data = pd.read_csv(z.open(agency_files[0]))
        agency_data.rename(columns= lambda x : x.lower(), inplace=True)
        agency_data.rename(columns={    
                                        "ncic_agency_name": "agency_name",    
                                        "ucr_agency_name" : "short_name",
                                        "primary_county"  : "county_name",
                                    },
                            inplace=True)
        agency_data.loc[:, "year"] = [int(zip_file[-8:-4])] * len(agency_data)
        if len(agency_data) > 0:
            agencies = agencies.append(agency_data)



 IA 
 ID 
 SC 
 TN 
 VA 
 VT 

In [125]:
columns = ["state_abbr", "year", "agency_id", "ori", "legacy_ori", "agency_name", "short_name", "population", "county_name"]

agencies = agencies[columns] 

In [147]:
import requests
from bs4 import BeautifulSoup

url = "https://www.icpsr.umich.edu/files/NACJD/ORIs"

page = requests.get(url)

soup = BeautifulSoup(page.content, "html.parser")

state_url = [url + "/" + i.get("href") for i in soup.find_all("a") if "oris" in i.get("href")]

In [178]:
data = pd.DataFrame()

page = requests.get(state_url[0])
soup = BeautifulSoup(page.content, "html.parser")

for pre in  soup.find_all("pre"):

    content = pre.text.strip().split("\n")
    content = [c for c in content if len(c) > 0]
    content = [ [cc.strip() for cc in c.split("  ") if len(cc) > 1] for c in content]

    data = data.append(pd.DataFrame({
                    "state" : [state] * len(content),
                    content[0][0]: [cc[0] for cc in content[1:]],
                    content[0][1]: [cc[1] for cc in content[1:]],
                    content[0][2]: [cc[2] for cc in content[1:]],
                }
            )
    )

data

,CITY/AGENCY,ORI7,ORI9
0,AUTAUGA COUNTY SHERIFF'S OFFICE,AL00400,AL0040000
1,AUTAUGAVILLE POLICE DEPARTMENT,AL00402,AL0040200
2,PRATTVILLE POLICE DEPARTMENT,AL00401,AL0040100
0,BALDWIN COUNTY SHERIFF'S OFFICE,AL00500,AL0050000
1,BAY MINETTE POLICE DEPARTMENT,AL00501,AL0050100
...,...,...,...
1,ARLEY POLICE DEPARTMENT,AL06704,AL0670400
2,DOUBLE SPRINGS POLICE DEPARTMENT,AL06701,AL0670100
3,HALEYVILLE POLICE DEPARTMENT,AL06702,AL0670200
4,TOWN OF LYNN POLICE DEPARTMENT,AL06705,AL0670500


In [179]:
state_url

['https://www.icpsr.umich.edu/files/NACJD/ORIs/01oris.html',
 'https://www.icpsr.umich.edu/files/NACJD/ORIs/02oris.html',
 'https://www.icpsr.umich.edu/files/NACJD/ORIs/04oris.html',
 'https://www.icpsr.umich.edu/files/NACJD/ORIs/05oris.html',
 'https://www.icpsr.umich.edu/files/NACJD/ORIs/06oris.html',
 'https://www.icpsr.umich.edu/files/NACJD/ORIs/08oris.html',
 'https://www.icpsr.umich.edu/files/NACJD/ORIs/09oris.html',
 'https://www.icpsr.umich.edu/files/NACJD/ORIs/10oris.html',
 'https://www.icpsr.umich.edu/files/NACJD/ORIs/11oris.html',
 'https://www.icpsr.umich.edu/files/NACJD/ORIs/12oris.html',
 'https://www.icpsr.umich.edu/files/NACJD/ORIs/13oris.html',
 'https://www.icpsr.umich.edu/files/NACJD/ORIs/15oris.html',
 'https://www.icpsr.umich.edu/files/NACJD/ORIs/16oris.html',
 'https://www.icpsr.umich.edu/files/NACJD/ORIs/17oris.html',
 'https://www.icpsr.umich.edu/files/NACJD/ORIs/18oris.html',
 'https://www.icpsr.umich.edu/files/NACJD/ORIs/19oris.html',
 'https://www.icpsr.umic

In [180]:

url = "https://www.icpsr.umich.edu/files/NACJD/ORIs"

page = requests.get(url)

soup = BeautifulSoup(page.content, "html.parser")

In [181]:
soup

<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 3.2 Final//EN">

<html>
<head>
<title>Index of /files/NACJD/ORIs</title>
</head>
<body>
<h1>Index of /files/NACJD/ORIs</h1>
<pre><img alt="Icon " src="/icons/blank.gif"/> <a href="?C=N;O=D">Name</a>                    <a href="?C=M;O=A">Last modified</a>      <a href="?C=S;O=A">Size</a>  <hr/><img alt="[DIR]" src="/icons/back.gif"/> <a href="/files/NACJD/">Parent Directory</a>                             -   
<img alt="[TXT]" src="/icons/text.gif"/> <a href="01oris.html">01oris.html</a>             17-Dec-2015 14:36   55K  
<img alt="[TXT]" src="/icons/text.gif"/> <a href="02oris.html">02oris.html</a>             17-Dec-2015 14:36   11K  
<img alt="[TXT]" src="/icons/text.gif"/> <a href="04oris.html">04oris.html</a>             17-Dec-2015 14:36   18K  
<img alt="[TXT]" src="/icons/text.gif"/> <a href="05oris.html">05oris.html</a>             17-Dec-2015 14:36   44K  
<img alt="[TXT]" src="/icons/text.gif"/> <a href="06oris.html">06oris.html</a> 